In [ ]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import glob
import xlsxwriter

# 📌 Function to load MRI images
def load_mri_images(directory, probe):
    """
    Loads MRI images from a specified directory and converts them to numpy arrays.
    
    📌 Filename Structure:
    - 'p1'  → 3FDG Probe
    - 'p2'  → 3FDGal Probe
    - 't1'  → 1 hour after injection
    - 't2'  → 2 hours after injection
    - 't3'  → 3 hours after injection
    - 'm1'  → First metabolite (Glucose/Galactose)
    - 'm3'  → Second metabolite (Gluconic Acid/Galactonic Acid)
    - 'm4'  → Third metabolite (Sorbitol/Galactitol)

    Parameters:
    - directory (str): The path where MRI images are stored.
    - probe (str): The probe type ('p1' for 3FDG, 'p2' for 3FDGal).

    Returns:
    - A dictionary containing image filenames as keys and numpy image arrays as values.
    """
    if not os.path.exists(directory):
        print(f"❌ ERROR: The directory {directory} does not exist! Please check the path.")
        return {}

    images = {}
    probe_prefix = "711" if probe == "p2" else "712"  # 711 → 3FDGal, 712 → 3FDG

    for file_name in os.listdir(directory):
        if file_name.startswith(probe_prefix) and file_name.endswith(".tif"):
            file_path = os.path.join(directory, file_name)
            try:
                img = Image.open(file_path)
                img_array = np.array(img)
                images[file_name] = img_array
            except Exception as e:
                print(f"❌ ERROR loading {file_path}: {e}")

    if not images:
        print(f"⚠ No images found for probe {probe.upper()} in {directory}!")

    return images

# 📌 Function to compute SNR
def compute_snr(image_data):
    max_signal = np.max(image_data)
    noise_std = np.std(image_data)  # Assuming noise can be estimated as the std of the entire image
    snr = max_signal / noise_std if noise_std != 0 else 0
    return snr, max_signal, noise_std, image_data

# 📌 Function to save SNR results to an Excel file
def save_snr_to_excel(images, probe, repo_directory):
    """
    Computes and saves SNR values into an Excel file, organized by probe type.
    """
    snr_results_dir = os.path.join(repo_directory, "snr_results")
    os.makedirs(snr_results_dir, exist_ok=True)

    metabolite_map = {
        "p1": {"m1": "3FDGlucose", "m3": "3FDGluconic Acid", "m4": "3FDSorbitol"},
        "p2": {"m1": "3FDGalactose", "m3": "3FDGalactonic Acid", "m4": "3FDGalactitol"}
    }
    time_map = {"t1": "1hr", "t2": "2hr", "t3": "3hr"}

    summary_data = []

    for file_path, img in images.items():
        snr, max_signal, noise, _ = compute_snr(np.array(img))

        parts = file_path.replace(".tif", "").split("_")
        if len(parts) < 4:
            continue  # Skip unexpected formats

        time_label = time_map.get(parts[2], parts[2])
        metabolite_label = metabolite_map[probe].get(parts[3], parts[3])

        summary_data.append([file_path, metabolite_label, time_label, snr, max_signal, noise])

    output_excel = os.path.join(snr_results_dir, f"SNR_Results_{probe.upper()}.xlsx")
    df = pd.DataFrame(summary_data, columns=["Filename", "Metabolite", "Time", "SNR", "Max Signal", "Noise Std Dev"])
    
    with pd.ExcelWriter(output_excel, engine="xlsxwriter") as writer:
        df.to_excel(writer, sheet_name="SNR_Data", index=False)

    print(f"SNR results saved successfully: {output_excel}")

# 📝 Define repository path
repo_directory = r"YOUR_LOCAL_PATH_HERE/FMRI-Metabolism-Quantification-main"  # ← UPDATE THIS PATH!
example_images_dir = os.path.join(repo_directory, "example_images_3FDGal")  # Change to "example_images_3FDG" for 3FDG
probe = "p2"  # "p1" for 3FDG, "p2" for 3FDGal

# 📂 Load images before calling save_snr_to_excel
images = load_mri_images(example_images_dir, probe)

# ✅ Check if images loaded correctly
if not images:
    raise ValueError("❌ ERROR: No images were loaded! Check the directory path and probe type.")

# 🔍 Compute and save SNR results to Excel
save_snr_to_excel(images, probe, repo_directory)
